In [6]:
import json, sys
import pandas as pd


def load_dict_from_file(file_path):
    """ Load JSON dictionary from a file. """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)['f']
    except FileNotFoundError:
        print("File not found.")
        sys.exit(1)
    except json.JSONDecodeError:
        print("Error decoding JSON.")
        sys.exit(1)


def parse_non_covered_lines(coverage_dict):
    all_non_covered_lines = {}

    for file, coverage_data in coverage_dict.items():
        # Parse percentage
        file_coverage_info = {'percentage': coverage_data['p']}

        # Make a set for the non-covered lines
        non_covered_lines = set()
        raw_non_covered_lines = coverage_data['u']

        for entry in raw_non_covered_lines:
            # index 0: start line
            start_line = entry[0]

            # index 2: end line
            end_line = entry[2]

            # add all numbers between start and end line inclusive to set
            non_covered_lines.update(range(start_line, end_line + 1))

        # Add all the lines numbers (inclusive)
        file_coverage_info['non_covered_lines'] = non_covered_lines
        all_non_covered_lines[file] = file_coverage_info

    return all_non_covered_lines


# def parse_covered_lines(coverage_dict):
#     all_covered_lines = {}
#
#     for file, coverage_data in coverage_dict.items():
#         file_coverage_info = {'percentage': coverage_data['p']}
#         # Make a set for the non-covered lines
#
#         covered_lines = set()
#         raw_covered_lines = coverage_data['g']
#
#         for entry in raw_covered_lines:
#             covered_lines.update(entry['s'])
#
#         file_coverage_info['covered_lines'] = covered_lines
#         all_covered_lines[file] = file_coverage_info
#
#     return all_covered_lines


# def compare_coverage(fuzzer_non_covered_lines, cts_non_covered_lines):
#     coverage_diff = {}
#     for file, coverage in fuzzer_non_covered_lines.items():
#         lines_not_covered_by_fuzzer = fuzzer_non_covered_lines[file]['uncovered_lines']
#
#         if (file in cts_non_covered_lines):
#             lines_not_covered_by_cts = cts_non_covered_lines[file]['uncovered_lines']
#         else:
#             lines_not_covered_by_cts = set()
#
#         lines_covered_by_fuzzer_and_not_cts = lines_not_covered_by_cts.difference(lines_not_covered_by_fuzzer)
#
#     return coverage_diff


def compare_coverage(fuzzer_non_covered_lines, cts_non_covered_lines):
    coverage_diff = {}

    for file, coverage_data in fuzzer_non_covered_lines.items():
        fuzzer_percent_coverage = coverage_data['percentage']

        if file not in cts_non_covered_lines:
            cts_percent_coverage = 0
            lines_not_covered_by_cts = set()
        else:
            cts_percent_coverage = cts_non_covered_lines[file]['percentage']
            lines_not_covered_by_cts = cts_non_covered_lines[file]['non_covered_lines']

        file_coverage_info = {'fuzzer_percentage': fuzzer_percent_coverage, 'cts_percentage': cts_percent_coverage}

        lines_not_covered_by_fuzzer = coverage_data['non_covered_lines']


        lines_covered_by_fuzzer_but_not_cts = lines_not_covered_by_cts.difference(lines_not_covered_by_fuzzer)

        file_coverage_info['num_lines_missed'] = len(lines_covered_by_fuzzer_but_not_cts)
        file_coverage_info['lines_missed'] = sorted(lines_covered_by_fuzzer_but_not_cts)
        coverage_diff[file] = file_coverage_info

    return coverage_diff


def display_difference(difference_in_coverage):
    # Create a list of tuples from the dictionary, selecting both 'percentage' and 'difference' values
    data = [(file, info['fuzzer_percentage'], info['cts_percentage'], info['num_lines_missed'], info['lines_missed']) for file, info in difference_in_coverage.items()]

    # Create DataFrame with appropriate column names
    df = pd.DataFrame(data, columns=['File', 'Fuzzer coverage (%)', 'CTS coverage (%)', 'Num lines missed', 'Lines missed'])
    
    filtered_df = df[df['Num lines missed'] > 0]

    # Sort the DataFrame by the 'File' column
    average_coverage_fuzzer = df['Fuzzer coverage (%)'].median()
    average_coverage_cts = df['CTS coverage (%)'].median()
    print("Median fuzzer coverage:", average_coverage_fuzzer)
    print("Medina CTS coverage:", average_coverage_cts)
    return filtered_df.sort_values(by='File')


def collate_coverage(dict_to_collate_to, dict_to_collate_from):
    for file, coverage_data in dict_to_collate_from.items():
        # Add file to merged dictionary
        if file not in dict_to_collate_to:
            dict_to_collate_to[file] = coverage_data
            continue

        # File present in both dictionaries so need to merge the data
        # percent_coverage_to_merge = coverage_data['percentage']
        non_covered_lines_to_merge = coverage_data['non_covered_lines']
        collated_non_covered_lines = dict_to_collate_to[file]['non_covered_lines'].intersection(non_covered_lines_to_merge)
        
        dict_to_collate_to[file]['non_covered_lines'] = collated_non_covered_lines


def cov_compare(fuzzer_cov_path, cts_cov_path):
    # Get coverage stats from JSON
    fuzzer_coverage_dict = load_dict_from_file(fuzzer_cov_path)
    cts_coverage_dict = load_dict_from_file(cts_cov_path)
    cts_shader_coverage_dict = load_dict_from_file(cts_shader_coverage_path)

    # Parse all covered lines by file
    fuzzer_non_covered_lines = parse_non_covered_lines(fuzzer_coverage_dict)
    cts_non_covered_lines = parse_non_covered_lines(cts_coverage_dict)
    cts_shader_non_covered_lines = parse_non_covered_lines(cts_shader_coverage_dict)

    collate_coverage(cts_non_covered_lines, cts_shader_non_covered_lines)

    # Get a dictionary of the percent covered by fuzzer but not CTS
    difference_in_coverage = compare_coverage(fuzzer_non_covered_lines, cts_non_covered_lines)

    # Print to dataframe
    return display_difference(difference_in_coverage)


# File paths (replace these with the actual paths to your JSON files)
fuzzer_coverage_path = './webglitch_swarm_formatted.json'
cts_coverage_path = './cts_api_formatted.json'
cts_shader_coverage_path = './cts_shader_formatted.json'

df = cov_compare(fuzzer_coverage_path, cts_coverage_path)
df


Average fuzzer coverage: 0.1578947368421053
Average CTS coverage: 0.24626865671641796


,File,Fuzzer coverage (%),CTS coverage (%),Num lines missed,Lines missed
4,out/Debug/gen/node/src/dawn/node/interop/WebGP...,0.115220,0.246927,14,"[872, 1912, 1913, 1916, 2831, 2899, 2900, 2901..."
89,src/dawn/native/CommandBuffer.cpp,0.140845,0.394366,4,"[72, 73, 74, 75]"
104,src/dawn/native/EncodingContext.cpp,0.451456,0.470874,5,"[175, 177, 178, 179, 180]"
124,src/dawn/native/ObjectBase.cpp,0.347458,0.372881,3,"[127, 128, 129]"
155,src/dawn/native/Texture.cpp,0.328656,0.413589,5,"[864, 865, 1331, 1332, 1333]"
194,src/dawn/node/binding/GPU.cpp,0.292419,0.314079,2,"[272, 273]"
195,src/dawn/node/binding/GPUAdapter.cpp,0.471861,0.640693,3,"[129, 130, 131]"
209,src/dawn/node/binding/GPUDevice.cpp,0.280000,0.416364,5,"[160, 161, 162, 163, 164]"
212,src/dawn/node/binding/GPUQueue.cpp,0.288136,0.474576,1,[157]
374,src/tint/lang/wgsl/ast/transform/promote_side_...,0.429984,0.509061,3,"[159, 160, 161]"
